In [1]:
pip install mwclient

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Ethereum'] #Ethereum page

In [3]:
revs = list(page.revisions()) #page revision , big list of edits

In [4]:
revs[0]

OrderedDict([('revid', 1218713785),
             ('parentid', 1216972551),
             ('user', 'Filedelinkerbot'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=4, tm_mday=13, tm_hour=10, tm_min=52, tm_sec=52, tm_wday=5, tm_yday=104, tm_isdst=-1)),
             ('comment',
              'Bot: Removing [[Commons:File:Compound Finance.jpg]] ([[:File:Compound Finance.jpg|en]]). It was deleted on Commons by [[:Commons:User:Krd|Krd]] (No permission since 5 April 2024).')])

In [5]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [6]:
revs[0] #first entry

OrderedDict([('revid', 592567939),
             ('parentid', 0),
             ('user', 'Sanpitch'),
             ('timestamp',
              time.struct_time(tm_year=2014, tm_mon=1, tm_mday=27, tm_hour=1, tm_min=53, tm_sec=45, tm_wday=0, tm_yday=27, tm_isdst=-1)),
             ('comment',
              "[[WP:AES|←]]Created page with '{{Infobox currency | image_1 =  | image_title_1 =  | image_width_1 =  | image_2 =  | image_title_2 =  | image_width_2 =  |issuing_authority = None. The Ethereum...'")])

In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
print(torch.__version__)

2.2.0


Finding the sentiment of page edits

In [9]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"] #strengt of the sentiment
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

/Users/moritzrichter/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [11]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [12]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [13]:
edits_df

,edit_count,sentiment,neg_sentiment
2014-01-27,1,-0.998511,1.000000
2014-02-01,1,-0.997276,1.000000
2014-04-06,5,0.790980,0.000000
2014-04-09,24,0.646407,0.083333
2014-04-10,9,-0.361518,0.666667
...,...,...,...
2024-03-18,1,-0.998051,1.000000
2024-03-19,1,-0.999787,1.000000
2024-04-01,2,-0.999709,1.000000
2024-04-03,2,-0.010781,0.500000


In [14]:
edits_df.index = pd.to_datetime(edits_df.index)

In [15]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [16]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [17]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,0,0.00000,0.0
2009-03-09,0,0.00000,0.0
2009-03-10,0,0.00000,0.0
2009-03-11,0,0.00000,0.0
2009-03-12,0,0.00000,0.0
...,...,...,...
2024-04-11,0,0.00000,0.0
2024-04-12,0,0.00000,0.0
2024-04-13,1,-0.99972,1.0
2024-04-14,0,0.00000,0.0


In [18]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [19]:
rolling_edits = rolling_edits.dropna()

In [20]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.000000,0.000000,0.000000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-04-11,0.200000,-0.100278,0.116667
2024-04-12,0.200000,-0.100278,0.116667
2024-04-13,0.233333,-0.133602,0.150000
2024-04-14,0.233333,-0.133602,0.150000


In [21]:
rolling_edits.to_csv("wikipedia_edits_eth.csv")